# M49 GC/Shell Analysis

Code for Taylor et al. 2020 investigating whether clusters stripped from VCC 1249 are a plausible source of a dip in the velocity profile of M49's GC system.

M49 potential parameters from M. Bílek et al. 2019, using Cote 2003 GC data, distance from Tonry et al. SBC:

|  β  |  M/L|         d   |     log(ρ_s)|   log(r_s) |    c  | log(r_vir)| log(Mvir) |log(M∗)|
|----|:-----:|:-------------:|:-----------:|:-----------:|:------:|:---------:|:---------:|:--------:|
| iso | 5 ± 1  | 16.1 ± 0.7 |  5.7 ± 0.2 |  2.9 ± 0.2 |   4.9  |   3.6 |      15.4|     11.6| 
| neg | 5 ± 1  | 16.2 ± 0.7 |  5.7 ± 0.2 |  2.8 ± 0.3 |   4.8  |   3.5 |      15.3|     11.6| 
| lit | 5 ± 1  | 16.1 ± 0.7 |  5.8 ± 0.2 |  2.9 ± 0.2 |   5.0  |   3.6 |      15.5|     11.6| 

Question these assumptions!


Kinematics from Battaia et al. 2012 and M.G. Lee et al. 1997

|Region| Proj.Dist.| Proj.Dist.| Velocity| X | Y |
|----|:-----:|:-----:|:-----:|:-----:|:-----:|
| | kpc|arcsec| km s-1| px| px|
| VCC 1249 | 0 | 0 | 390 ± 30 | | | 
| C12 | 0.5 | 5.8 | 390 ± 30| 414.48 | 382.30 |
| C17 | 1.5 | 19.2 | 390 ± 30| 442.68| 372.07|
| C6  | 5.6 | 70.7| 533 ± 53| 334.29| 510.38|
| (H i) | 10.0 | | 469 ± 3| | |
| C2 | 10.0 | 126.1 |561 ± 34|267.5 |606.19 |
| C4 | 11.0 | 137.9 |656 ± 73| 299.05|650.47 |
| C1 | 12.0 | 151.4 |716 ± 106|258.52 | 663.34|
| (M 49) | 26.5 | 334.0 | 1001 ± 12 | | |

In [210]:
%matplotlib widget
import os
import sys
import time
import numpy as np
import scipy
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import amuse.io
from amuse.lab import *
from amuse.datamodel import Particles
import amuse.plot
from amuse.community.bhtree.interface import BHTree
from amuse_helpers import initialize_nfw_model
from astropy.coordinates import SkyCoord
import astropy.units as u
from galpy.potential import NFWPotential
from galpy.potential import ChandrasekharDynamicalFrictionForce
from galpy.util import bovy_conversion, bovy_coords
from galpy.orbit import Orbit

#set environment variables for your directory structure in .bash_profile and .bash_rc, e.g.
#    export _M49_DATA_DIR=/path/to/M49data/
_DATADIR  =  os.environ['_M49_DATA_DIR']
_LOCALDIR =  os.environ['_M49_LOCAL_DIR']

## Observations

In [241]:
scVCC1249 = SkyCoord(l=287.140966*u.deg, b=70.141573*u.deg, radial_velocity=(390)*u.km/u.s, frame = 'galactic')
scM49 = SkyCoord(l=286.9218091*u.deg,  b=70.1960685*u.deg,radial_velocity=(1001)*u.km/u.s, frame = 'galactic')  
#assume C12 is at the center of VCC 1249 since it's unclear from the data
pxscale = 151.4/(np.sqrt((414.48-258.52)**2+(382.3-663.34)**2))#arcsec/px
physscale = 26.5/334 #kpc/arcsec
scHII = SkyCoord(
    ra=(np.array([414.48, 442.68, 334.29, 267.5, 299.05, 258.52])-414.48)*(pxscale)*u.arcsec+scVCC1249.icrs.ra,
    dec=(np.array([382.3,372.07,510.38,606.19,650.47,663.34])-382.3)*(pxscale)*u.arcsec+scVCC1249.icrs.dec, 
    radial_velocity=(np.array([390,390,533,561,656,716]))*u.km/u.s, frame='icrs')

im = plt.imread('M49.png')
llc = SkyCoord(ra='12h30m10s', dec='7d54m', frame='icrs')
urc = SkyCoord(ra='12h29m20s', dec='8d4m', frame='icrs')

plt.figure(figsize=(9,5))
plt.subplot(121, aspect='equal')
plt.imshow(im, extent = [llc.ra.degree, urc.ra.degree, llc.dec.degree, urc.dec.degree])
plt.scatter(scVCC1249.icrs.ra, scVCC1249.icrs.dec, c='r', s=40, label = 'VCC1249')
plt.scatter(scM49.icrs.ra, scM49.icrs.dec, c='b', s=50, label = 'M49')
plt.scatter(scHII.icrs.ra, scHII.icrs.dec, c='forestgreen', s=10, label = 'HII Regions')
plt.xlabel('RA')
plt.xlabel('Dec')
plt.legend(loc='upper right')

plt.subplot(122)
plt.scatter([0],scM49.radial_velocity, 
            c='b', s=50, label = 'M49')
plt.scatter(scM49.separation(scVCC1249).to(u.arcsec), scVCC1249.radial_velocity, 
            c='r', s=40, label = 'VCC1249')
plt.scatter(scM49.separation(scHII).to(u.arcsec), scHII.radial_velocity,
            c='forestgreen', s=10, label = 'HII Regions')
plt.xlabel('Radius [arcsec]')
plt.ylabel('Radial velocity [km/s]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Text(0,0.5,'Radial velocity [km/s]')

## Conversion to natural M49 system

In [284]:
R0,V0=8.,220.
rs = (scM49.separation(scHII).to(u.arcsec)*physscale).value
phis = scM49.position_angle(scHII).radian
vzs = (scHII.radial_velocity - scM49.radial_velocity).value
init_vxvv=[rs[0]/R0, 0., 0., 0., vzs[0]/V0, phis[0]]
vxvv = np.array([rs/R0, np.zeros(6), np.zeros(6), np.zeros(6), vzs/V0, phis]).T
vxvv_err = np.array([.1*np.ones(6), 3*np.ones(6), 3*np.ones(6), 3*np.ones(6), .1*np.ones(6), .05*np.ones(6)]).T
#iR,ivR,ivT,iz,ivz,iphi
init_orbit = Orbit(init_vxvv)
data_orbit = Orbit(vxvv)

## Fit orbit

In [315]:
nfw = NFWPotential(conc=4.8, mvir=10**15.4/1e12)
cdf = ChandrasekharDynamicalFrictionForce(GMs=1e13*u.Msun)
pot = nfw+cdf


fit_orbit = Orbit.from_fit(init_vxvv, vxvv, vxvv_err=vxvv_err, pot=pot)

## Plot in natural system

In [316]:
ts = np.linspace(0,.5,1000)*u.Gyr
mts = -ts

plt.figure(figsize=(9,5))
plt.subplot(121, aspect='equal')
plt.scatter([0], [0], c='b', s=50, label = 'M49')
plt.scatter(data_orbit.x(), data_orbit.y(), c='forestgreen', s=10, label = 'HII Regions')
plt.scatter(init_orbit.x(), init_orbit.y(), c='goldenrod', s=20, label = 'initial guess')
plt.scatter(fit_orbit.x(), fit_orbit.y(), c='maroon', s=20, label = 'fit orbit')
init_orbit.integrate(ts,pot=pot)
fit_orbit.integrate(ts,pot=pot)
plt.plot(init_orbit.x(ts), init_orbit.y(ts), c='goldenrod',alpha = 0.3)
plt.plot(fit_orbit.x(ts), fit_orbit.y(ts), c='maroon',alpha = 0.3)
init_orbit.integrate(mts,pot=pot)
fit_orbit.integrate(mts,pot=pot)
plt.plot(init_orbit.x(mts), init_orbit.y(mts), c='goldenrod',alpha = 0.3)
plt.plot(fit_orbit.x(mts), fit_orbit.y(mts), c='maroon',alpha = 0.3)
plt.xlabel('x [1]')
plt.ylabel('y [1]')
plt.xlim(-5,5)
plt.ylim(-5,5)
plt.legend(loc='upper right')

plt.subplot(122, aspect='equal')
plt.scatter([0], [0], c='b', s=50, label = 'M49')
plt.scatter(data_orbit.R(), data_orbit.vz(), c='forestgreen', s=10, label = 'HII Regions')
plt.scatter(init_orbit.R(), init_orbit.vz(), c='goldenrod', s=20, label = 'initial guess')
plt.scatter(fit_orbit.R(), fit_orbit.vz(), c='maroon', s=20, label = 'fit orbit')
init_orbit.integrate(ts,pot=pot)
fit_orbit.integrate(ts,pot=pot)
#plt.plot(init_orbit.R(ts), init_orbit.vz(ts), c='goldenrod',alpha = 0.3)
#plt.plot(fit_orbit.R(ts), fit_orbit.vz(ts), c='maroon',alpha = 0.3)
init_orbit.integrate(mts,pot=pot)
fit_orbit.integrate(mts,pot=pot)
plt.plot(init_orbit.R(mts), init_orbit.vz(mts), c='goldenrod',alpha = 0.3)
plt.plot(fit_orbit.R(mts), fit_orbit.vz(mts), c='maroon',alpha = 0.3)
plt.xlabel('Radius [1]')
plt.ylabel('vz [1]')
plt.xlim(-1,7)
plt.ylim(-4,4)
plt.legend(loc='upper right')

/Users/hendel/anaconda2/envs/py37/lib/python3.7/site-packages/matplotlib/pyplot.py:522: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

## Find inital conditions for simulation

In [335]:
ts = np.linspace(0,-2.77,3000)*u.Gyr
fit_orbit.turn_physical_on()
fit_orbit.integrate(ts,pot=pot)
fit_orbit.plot3d()
fit_orbit.plot(d1='t',d2='r')

#-2.77 Gyr seems to give a good apocenter

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

In [344]:
#get IC and double check it is going the right way
ic_orbit = fit_orbit(-2.77*u.Gyr)
#print(fit_orbit.vx(-2.77*u.Gyr),fit_orbit.vy(-2.77*u.Gyr),fit_orbit.vz(-2.77*u.Gyr))
print(ic_orbit.x(),ic_orbit.y(), ic_orbit.z())
print(ic_orbit.vx(),ic_orbit.vy(), ic_orbit.vz())
#icts = np.linspace(0,.1,100)*u.Gyr
#ic_orbit.integrate(icts, pot=pot)

#plt.figure()
#plt.scatter(fit_orbit.r(ts[-1]),fit_orbit.y(ts[-1]),s=100)
#plt.plot(fit_orbit.r(ts[-100:]),fit_orbit.y(ts[-100:]), lw=3)
#plt.scatter(ic_orbit.r(),ic_orbit.y())
#plt.plot(ic_orbit.r(icts),ic_orbit.y(icts))

43.1608085375 -66.563793623 -20.5541560126
43.1332558515 -96.5279993982 454.238347169


## Plot simulation

## <span style="color:red" style="font-size:50px">**not final!**</span>

In [235]:
stars, converter = initialize_nfw_model(1e5, mass = 1e12, scale_radius=25.)
eint = stars[:].potential(smoothing_length_squared=(0.1 | units.kpc)**2)

In [305]:
tf = amuse.io.read_set_from_file('./data/snap_2200.csv')
mask = tf[100000:][eint<np.percentile(eint,10)]

In [306]:
plt.figure()
plt.subplot(121, aspect='equal')
hhh = plt.hist2d(tf.x.value_in(units.kpc),tf.y.value_in(units.kpc),
                 bins=np.linspace(-100,100,200), norm=LogNorm(), cmap='gray')
plt.xlabel('x [kpc]')
plt.ylabel('y [kpc]')
plt.title('DM')
plt.subplot(122, aspect='equal')
hhh = plt.hist2d(mask.x.value_in(units.kpc),mask.y.value_in(units.kpc),
                 bins=np.linspace(-100,100,200), norm=LogNorm(), cmap='afmhot')
plt.xlabel('x [kpc]')
plt.ylabel('y [kpc]')
plt.title('Stars')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Text(0.5,1,'Stars')